In [3]:
images = filtered_dataset['image']

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import random

def random_crop_images(directory, images, crop_size=(512, 512)):
    # Check if the directory exists
    if not os.path.exists(directory):
        print(f"Directory {directory} does not exist.")
        return

    # Loop through all the images in the dataset
    for img in images:
        # Get the dimensions of the image
        img_width, img_height = img.size

        # Ensure the crop size is not larger than the image
        crop_width, crop_height = min(crop_size[0], img_width), min(crop_size[1], img_height)

        # Check if the image is smaller than the desired crop size
        if img_width < crop_width or img_height < crop_height:
            print("Image is too small for the desired crop size.")
            continue

        # Calculate random position for the crop
        left = random.randint(0, img_width - crop_width)
        upper = random.randint(0, img_height - crop_height)

        # Perform the crop
        cropped_img = img.crop((left, upper, left + crop_width, upper + crop_height))

        # Convert the cropped image to RGB (to remove alpha channel for JPEG format)
        cropped_img = cropped_img.convert('RGB')

        # Generate a new filename for the cropped image
        cropped_image_filename = 'cropped_image_' + str(random.randint(1, 999999))

        # Save the cropped image in JPEG format with optimization and reduced quality
        cropped_img.save(os.path.join(directory, cropped_image_filename), 'JPEG', optimize=True, quality=85)

        # Delete the original PNG image
        os.remove(os.path.join(directory, img))